In [1]:
import sys
import os
from  pathlib import  Path
sys.path.append(str(Path.cwd().parent))
from  config import OPENAI_API_KEY,NCBI_API_KEY,EMAIL
from  src.clinfoai.pubmed_engine import PubMedNeuralRetriever

/pasteur/u/ale9806/miniconda3/envs/clinfo/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Using Clinfo.AI 

In this tutorial, we will go through each step of the Clinfo.AI workflow. Before we start, we need to set up a few things. 


### 1.- Setting up enviorment:
1.a.- Install the conda enviroment using the yml file provided.

``` conda env create -f environment.yaml ```

1.b.- Select your environment to run notebook. I recommend using VScode: 



### 2.- Creating Accounts

You will need at least one account and at most two (depending on how many calls/hour you plan to do):
* OPENAI account: If you start a free account for the first time, you will get $5 in API credits.
* NCBI_API_KEY: This is only necessary if you plan to make more than 10 calls per hour.


Once you have created both accounts  go to **src\config.py** file and: 

* Set OPENAI_API_KEY to your openAI API key

If you created an NCBI API account add your key and email in the following values: 
* NCBI_API_KEY 
* EMAIL
* 
Otherwise, leave them as None

In [2]:
# Make Sure you followed at least step 1-2 before running this cell.
from  config import OPENAI_API_KEY, NCBI_API_KEY, EMAIL
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

### 3.- Defining your own promts:
We have designed prompts for each step of Clinfo.ai Workflow, leaveriging the power of in-contex-learning. If you want to us your own promps you can edit them **src\prompts** otherwise we will use the default prompts:

In [11]:
PROMPS_PATH = os.path.join("..","src","clinfoai","prompts","PubMed","Architecture_1","master.json")

### 4.- Define Clinfo.AI LLM Backbone
Clinfo uses a chain of LLMs to summarize information, thus we need to define an LLM backbone. 

We will start with OpenAI models, however, if you have access to GPUs it is possible to use Clinfo.AI with vLLM to use OpenSource LLMs as backbones (check tutorial 3).

In [12]:
MODEL:str  = "gpt-3.5-turbo"
#MODEL:str = "Qwen/Qwen2-beta-7B-Chat"

### 5.- Init Clinfo+Pubmed Engnie
We have all the necessary data to start our clinfo+pubmed instance:

In [13]:
## 5.- Init Neural Retriever from path. 
# Do not change the path if you want to use base  prompts, otherwise specify your own prompt architecture

nrpm = PubMedNeuralRetriever(
    architecture_path = PROMPS_PATH,
    model             = MODEL,
    verbose           = False,
    debug             = False,
    open_ai_key       = OPENAI_API_KEY,
    email             = EMAIL)


Task Name: pubmed_query_prompt
------------------------------------------------------------------------

Task Name: relevance_prompt
------------------------------------------------------------------------

Task Name: summarization_prompt
------------------------------------------------------------------------

Task Name: synthesize_prompt
------------------------------------------------------------------------


# Let's start!

In [14]:
### Step 0 : Ask a question ###
QUESTION    = "What is the prevalence of COVID-19 in the United States?"


## STEP 1 (Search PubMed): Convert the question into a query using an LLM
# This returns a list of queries (containing MESH terms)
# These queries are used to retrieve articles from NCBI
# Once retrieved we collect a list article ids.
pubmed_queries, article_ids = nrpm.search_pubmed(
    question=QUESTION,
    num_results=10,
    num_query_attempts=1)

print(f"Articles retrived: {len(article_ids)}")
print(pubmed_queries)
print(article_ids)

Articles retrived: 10
['prevalence COVID-19 United States']
['35206474', '34281357', '33481900', '34888288', '35996224', '34311990', '36508742', '33663642', '37639043', '34922126']


In [15]:
## Step 2: Fetch article data
# Preiously, we only extracted he PMIDs. No we will use those  PMIDs to retrive the metadata:
articles = nrpm.fetch_article_data(article_ids)

# Print example for first article: 
article_num = 1
print(f"Article {article_num}:\n")

#print(articles[article_num].keys())
#print(articles[article_num]['PubmedData'])
print(articles[article_num]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"])
#print(articles[article_num]["MedlineCitation"]["Article"])


Article 1:

['Myocarditis has been recognized as a rare complication of coronavirus disease 2019 (COVID-19) mRNA vaccinations, especially in young adult and adolescent males. According to the US Centers for Disease Control and Prevention, myocarditis/pericarditis rates are ≈12.6 cases per million doses of second-dose mRNA vaccine among individuals 12 to 39 years of age. In reported cases, patients with myocarditis invariably presented with chest pain, usually 2 to 3 days after a second dose of mRNA vaccination, and had elevated cardiac troponin levels. ECG was abnormal with ST elevations in most, and cardiac MRI was suggestive of myocarditis in all tested patients. There was no evidence of acute COVID-19 or other viral infections. In 1 case, a cardiomyopathy gene panel was negative, but autoantibody levels against certain self-antigens and frequency of natural killer cells were increased. Although the mechanisms for development of myocarditis are not clear, molecular mimicry between th

In [16]:
# STEP 3 Summarize each article
# This step is parallelized, though it might look like one single call, it performs one call per article to summarize.
# Then the relevancy of the article (based on the original question) is provided by another LLM call.

article_summaries,irrelevant_articles =  nrpm.summarize_each_article(articles, QUESTION)

In [17]:
# Summaries for relevant articles
article_summaries

[{'title': 'United States response to the COVID-19 pandemic, January-November 2020.',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/33663642/',
  'abstract': 'As of November 2020, the United States leads the world in confirmed coronavirus disease 2019 (COVID-19) cases and deaths. Over the past 10 months, the United States has experienced three peaks in new cases, with the most recent spike in November setting new records. Inaction and the lack of a scientifically informed, unified response have contributed to the sustained spread of COVID-19 in the United States. This paper describes major events and findings from the domestic response to COVID-19 from January to November 2020, including on preventing transmission, COVID-19 testing and contact tracing, ensuring sufficient physical infrastructure and healthcare workforce, paying for services, and governance. We further reflect on the public health response to-date and analyse the link between key policy decisions (e.g. closing, reopening) a

In [18]:
# Articles deemed irelevant
irrelevant_articles 

[{'title': 'Myocarditis With COVID-19 mRNA Vaccines.',
  'url': 'https://pubmed.ncbi.nlm.nih.gov/34281357/',
  'abstract': 'Myocarditis has been recognized as a rare complication of coronavirus disease 2019 (COVID-19) mRNA vaccinations, especially in young adult and adolescent males. According to the US Centers for Disease Control and Prevention, myocarditis/pericarditis rates are ≈12.6 cases per million doses of second-dose mRNA vaccine among individuals 12 to 39 years of age. In reported cases, patients with myocarditis invariably presented with chest pain, usually 2 to 3 days after a second dose of mRNA vaccination, and had elevated cardiac troponin levels. ECG was abnormal with ST elevations in most, and cardiac MRI was suggestive of myocarditis in all tested patients. There was no evidence of acute COVID-19 or other viral infections. In 1 case, a cardiomyopathy gene panel was negative, but autoantibody levels against certain self-antigens and frequency of natural killer cells were

In [20]:
# STEP 4 do a synthesis of all summaries to answer question: 
synthesis =   nrpm.synthesize_all_articles(article_summaries, QUESTION)
print("synthesis")
print(synthesis)

synthesis
Literature Summary: Studies with the strongest evidence suggest that the prevalence of COVID-19 in the United States varied over time, with factors such as policy decisions, vaccination campaigns, and the introduction of new variants influencing transmission rates. A systematic review reported COVID-19 vaccine acceptance rates ranging from 12% to 91.4% in the US population, with hesitancy highest among certain demographic groups. Modeling studies indicated that the vaccination campaign effectively reduced COVID-19 deaths, with an estimated infection fatality ratio of approximately 0.6-0.7%. Additionally, a community-based survey found a 1.7% positive rate for SARS-CoV-2 infection among US adults during a specific timeframe, with common symptoms including dry cough, fever, and shortness of breath. Children with COVID-19 generally experience milder symptoms and are at lower risk of severe complications, though rare cases of severe disease have been reported. Lastly, a populatio

# Great! We answered our first question using Clinfo.AI!
## Here are all the steps condensed:

In [16]:
PROMPS_PATH  = os.path.join("..","src","clinfoai","prompts","PubMed","Architecture_1","master.json")
MODEL:str    = "gpt-3.5-turbo"
MODEL:str    = "Qwen/Qwen2-beta-7B-Chat"

nrpm = PubMedNeuralRetriever(
    architecture_path = PROMPS_PATH,
    model             = MODEL,
    verbose           = False,
    debug             = False,
    open_ai_key       = OPENAI_API_KEY,
    email             = EMAIL)

### STEP 0: Ask a question ###
question    = "What is the prevalence of COVID-19 in the United States?"

### STEP 1: Search PubMed ###
pubmed_queries, article_ids = nrpm.search_pubmed(question,num_results=10,num_query_attempts=1)

### STEP 2: Fetch article data ###
articles = nrpm.fetch_article_data(article_ids)

### STEP 3 Summarize each article (only if they are relevant [Step 3]) ###
article_summaries,irrelevant_articles =  nrpm.summarize_each_article(articles, question)


### STEP 4: Synthesize the results ###
synthesis =   nrpm.synthesize_all_articles(article_summaries, question)
print("Result:")
print(synthesis)


Task Name: pubmed_query_prompt
------------------------------------------------------------------------

Task Name: relevance_prompt
------------------------------------------------------------------------

Task Name: summarization_prompt
------------------------------------------------------------------------

Task Name: synthesize_prompt
------------------------------------------------------------------------
Trying to init model via VLM
Result:
Literature Summary: The prevalence of COVID-19 in the United States varies across different populations and occupational groups. A cross-sectional study by Koh (2020) estimated that workers in the healthcare and social assistance industry, as well as healthcare professions and support roles, had elevated risks, with a national prevalence of around 1.7% in a community-based survey in 2020. A meta-analysis of hospitalized patients by Esper et al. (2021) found a prevalence of 6.9% with bacterial co-infection or secondary infection, with higher 